# Biblioteca

In [ ]:
!pip install selenium webdriver-manager
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

def fn_formata_dados(df,colunas_prct):
  # 1️⃣ Colunas numéricas comuns (não porcentagem)
  for col in df.columns:
      if col not in colunas_prct:
          df[col] = (
              df[col]
              .astype(str)
              .str.replace(",", ".", regex=False)  # vírgula → ponto
              .str.strip()                          # remove espaços extras
          )
          df[col] = pd.to_numeric(df[col], errors='coerce')

  # 2️⃣ Colunas de porcentagem
  for col in colunas_prct:
      if col in df.columns:
          df[col] = (
              df[col]
              .astype(str)
              .str.replace("%", "", regex=False)  # remove %
              .str.replace(",", ".", regex=False) # vírgula → ponto
              .str.strip()
          )
          df[col] = pd.to_numeric(df[col], errors='coerce') / 100

# **************  funcoes de webscrpping *******************
def fn_le_indicadores_web(ativo):
  print("Lendo " + ativo)
  url ='https://investidor10.com.br/acoes/' + ativo
  headers = {"User-Agent": "Mozilla/5.0"}

  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")

  indicadores =  {"Ativo": ativo}

  # Cada indicador está dentro de uma div com class "cell"
  for cell in soup.find_all("div", class_="cell"):
      nome_tag = cell.find("span")  # nome do indicador (ROE, P/L, etc.)
      valor_tag = cell.find("div", class_="value")

      if nome_tag and valor_tag:
          nome = nome_tag.get_text(strip=True).split(" ")[0]
          valor = valor_tag.find("span").get_text(strip=True)
          indicadores[nome] = valor

  # Add Setor and Segmento
  root = soup.select_one("#info_about")
  container = (root.select_one("#table-indicators-company") if root else None) \
              or soup.select_one("#table-indicators-company") \
              or soup  # fallback

  kv = {}
  for cell in container.select(".cell"):
      t = cell.select_one(".title")
      v = cell.select_one(".value .simple-value") or cell.select_one(".value")
      if t and v:
          kv[norm(t.get_text())] = norm(v.get_text(" "))

  def pick(*alts):
    # tenta direto, depois faz busca case-insensitive/prefixo
    for a in alts:
        if a in kv: return kv[a]
    folded = {norm(k).casefold(): k for k in kv}
    for a in alts:
        key = norm(a).casefold()
        if key in folded: return kv[folded[key]]
        for fk, orig in folded.items():
            if fk.startswith(key): return kv[orig]

  indicadores["Setor"] = pick("Setor")
  indicadores["Segmento"] = pick("Segmento")


  # retorna dicionário com os indicadores
  return indicadores

def norm(s): return re.sub(r"\s+", " ", s).strip()


def fn_altamente_correlacionados(p_matriz, p_cutoff):
  altamente_correlacionados = set()

  corr_matrix_abs = p_matriz.abs()
  corr_matrix_upper_abs = p_matriz.where(np.triu(np.ones(p_matriz.shape), k=1).astype(bool))

  while True:
      # Find the highest correlation exceeding the cutoff
      max_corr = corr_matrix_upper_abs.max().max()
      if max_corr < p_cutoff:
          break # No more correlations above cutoff

      # Find the pair with this highest correlation
      row_idx, col_idx = np.where(corr_matrix_upper_abs == max_corr)
      # Get the actual column names (handle potential multiple pairs with same max corr)
      col1 = corr_matrix_upper_abs.index[row_idx[0]]
      col2 = corr_matrix_upper_abs.columns[col_idx[0]]

      # Calculate mean absolute correlation for each column in the pair
      mean_abs_corr1 = corr_matrix_abs[col1].mean()
      mean_abs_corr2 = corr_matrix_abs[col2].mean()

      # Add the column with the higher mean absolute correlation to the set to remove
      if mean_abs_corr1 > mean_abs_corr2:
          col_to_remove = col1
      else:
          col_to_remove = col2 # If equal, remove the second one

      altamente_correlacionados.add(col_to_remove)

      # Remove the flagged column from consideration by setting its correlations to 0
      corr_matrix_upper_abs.loc[col_to_remove, :] = 0
      corr_matrix_upper_abs.loc[:, col_to_remove] = 0

  return altamente_correlacionados

def fn_comparar_histogramas(p_todos,  p_tratados):
  # exibem 2 histograma, uma antes e outro após a limpeza
  fig, axes = plt.subplots(1, 2, figsize=(12, 5))
  p_todos.hist(ax=axes[0])
  p_tratados.hist(ax=axes[1])
  plt.show()

def tratar_outliers_IQR(series, fator=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - fator * IQR
    limite_superior = Q3 + fator * IQR

    # Winsorização
    serie_tratada = series.clip(limite_inferior, limite_superior)

    # tratar ativos que não possuem alguns indicadores, substituindo pela mediana
    # ev_ebitda : 'BBSE3', 'BBDC3', 'BBDC4', 'BBAS3', 'BPAC11', 'CXSE3', 'IRBR3', 'ITUB4',   'SANB11'
    # div_brt_patr: 'BBDC3', 'BBDC4', 'BBAS3', 'BPAC11', 'ITUB4', 'SANB11'
    # cagr : 'AURE3', 'RAIZ4', 'SMFT3'
    mediana = serie_tratada.median()
    serie_tratada = serie_tratada.fillna(mediana)

    return serie_tratada

def normalizar_coluna(serie, is_melhor_maior=True):
    min_val,max_val = serie.min(), serie.max()

    if min_val == max_val: # Evita divisão por zero se todos os valores forem iguais
        return pd.Series(0.5, index=serie.index)

    if is_melhor_maior:
        return (serie - min_val) / (max_val - min_val)
    else:
        return (max_val - serie) / (max_val - min_val)

# Download de Indicadores

Serão utilizados somente os ativos que compões o IBOVESPA de modo a garantir liquidez. Fonte: https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm

Serão filtrados ativos descorrelacionados

Serão utilizados dados fundamentalista para filtragem: Maiores lucros, ROE ou margens financeirassua liquidez

In [ ]:
ativos_ibov = ["ALOS3", "ABEV3", "ASAI3", "AURE3", "AZZA3", "B3SA3", "BBSE3", "BBDC3",
    "BBDC4", "BRAP4", "BBAS3", "BRKM5", "BRAV3", "BRFS3", "BPAC11", "CXSE3",
    "CMIG4", "COGN3", "CPLE6", "CSAN3", "CPFE3", "CMIN3", "CVCB3", "CYRE3",
    "DIRR3", "ELET3", "ELET6", "EMBR3", "ENGI11", "ENEV3", "EGIE3", "EQTL3",
    "FLRY3", "GGBR4", "GOAU4", "HAPV3", "HYPE3", "IGTI11", "IRBR3", "ISAE4",
    "ITSA4", "ITUB4", "KLBN11", "RENT3", "LREN3", "MGLU3", "POMO4", "MRFG3",
    "BEEF3",  "MRVE3", "MULT3", "NTCO3", "PCAR3", "PETR3", "PETR4",
    "RECV3", "PRIO3", "PETZ3", "PSSA3", "RADL3", "RAIZ4", "RDOR3", "RAIL3",
    "SBSP3", "SANB11", "STBP3", "SMTO3", "CSNA3", "SLCE3", "SMFT3", "SUZB3",
    "TAEE11", "VIVT3", "TIMS3", "TOTS3", "UGPA3", "USIM5", "VALE3", "VAMO3",
    "VBBR3", "VIVA3", "WEGE3", "YDUQ3"]

linhas = [fn_le_indicadores_web(ativo) for ativo in ativos_ibov]

df_ind_fund = pd.DataFrame(linhas)
df_ind_fund.to_csv('indi_fundaamentalistas.csv', index=False)

# Filtragem de indicadores

In [17]:
df_ind_fund_raw = pd.read_csv('indi_fundaamentalistas.csv')
df_ind_fund_raw.set_index('Ativo' , inplace=True)
fn_formata_dados(df_ind_fund_raw, ['DIVIDEND','PAYOUT', 'MARGEM', 'ROE','ROIC','ROA', 'CAGR'])

indicadores extraídos: 'P/L', 'P/RECEITA', 'P/VP', 'DIVIDEND', 'PAYOUT', 'MARGEM', 'EV/EBITDA',
       'EV/EBIT', 'P/EBITDA', 'P/EBIT', 'P/ATIVO', 'P/CAP.GIRO', 'VPA', 'LPA',
       'GIRO', 'ROE', 'ROIC', 'ROA', 'DÍVIDA', 'PATRIMÔNIO', 'PASSIVOS',
       'LIQUIDEZ', 'CAGR'

A cada par de indicadores  que possuem correlação maior que 0.8 é excluído aqule  que tem maior correlação com os demais indicadores.

São eles: 'P/EBITDA', 'P/RECEITA', 'PATRIMÔNIO'

In [18]:
mat_corr_raw = df_ind_fund_raw.corr()
'''
plt.figure(figsize=(12,10))
sns.heatmap(mat_corr_raw, annot=True, fmt=".2f", cmap="coolwarm")
plt.show()
'''
df_ind_fund = df_ind_fund_raw.drop(columns = fn_altamente_correlacionados(mat_corr_raw, 0.8))
'''
mat_corr = df_ind_fund.corr()
plt.figure(figsize=(12,10))
sns.heatmap(mat_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.show()
'''

'\nmat_corr = df_ind_fund.corr()\nplt.figure(figsize=(12,10))\nsns.heatmap(mat_corr, annot=True, fmt=".2f", cmap="coolwarm")\nplt.show()\n'

Analisar:

 'P/L', 'P/VP', 'DIVIDEND', 'PAYOUT', 'MARGEM', 'EV/EBITDA', 'EV/EBIT',
       'P/EBIT', 'P/ATIVO', 'P/CAP.GIRO', 'VPA', 'LPA', 'GIRO', 'ROE', 'ROIC',
       'ROA', 'DÍVIDA', 'PATRIMÔNIO', 'PASSIVOS', 'LIQUIDEZ', 'CAGR'

Não foi considerado :DIVIDEND,PAYOUT  

De forma aidentificar outliers poderia usar o Método do Z-score ou Método do Intervalo Interquartil (mais robusta para dados que não seguem uma distribuição norma).

E para tratamento pode ser usado a simples remoção ou a Winsorization.

Foi escolhido Método do Intervalo Interquartil com Winsorization, pois evita a exclusão de um ativo que pode ser importante por outros motivos.

In [19]:
# APLICAR INTERVALO INTERQUATIL E WINSORIZATION
indice = {
    'pl': tratar_outliers_IQR(df_ind_fund['P/L']),
    'p_vp': tratar_outliers_IQR(df_ind_fund['P/VP']),
    'margem_ebitda': tratar_outliers_IQR(df_ind_fund['MARGEM']),
    'ev_ebitda': tratar_outliers_IQR(df_ind_fund['EV/EBITDA']),
    'ev_ebit': tratar_outliers_IQR(df_ind_fund['EV/EBIT']),
    'p_ebit': tratar_outliers_IQR(df_ind_fund['P/EBIT']),
    'preco_ativo_circ': tratar_outliers_IQR(df_ind_fund['P/ATIVO']),
    'preco_cap_giro': tratar_outliers_IQR(df_ind_fund['P/CAP.GIRO']),
    'vpa': tratar_outliers_IQR(df_ind_fund['VPA']),
    'lpa': tratar_outliers_IQR(df_ind_fund['LPA']),
    'giro': tratar_outliers_IQR(df_ind_fund['GIRO']),
    'roe': tratar_outliers_IQR(df_ind_fund['ROE']),
    'roic': tratar_outliers_IQR(df_ind_fund['ROIC']),
    'roa': tratar_outliers_IQR(df_ind_fund['ROA']),
    'div_brt_patr': tratar_outliers_IQR(df_ind_fund['DÍVIDA']),
    'pat_liq_ativ': tratar_outliers_IQR(df_ind_fund['PATRIMÔNIO']),
    'passivo_ativos': tratar_outliers_IQR(df_ind_fund['PASSIVOS']),
    'liquidez': tratar_outliers_IQR(df_ind_fund['LIQUIDEZ']),
    'cagr_lucros': tratar_outliers_IQR(df_ind_fund['CAGR'])
}

df_tratados = pd.DataFrame(indice)

# ATRIBUIR PESOS E NORMALIZAR INDICADORES
    # As colunas VPA, LPA, GIRO e PASSIVOS não serão usadas na pontuação,
    # pois sua informação já está contida em outros indicadores.

pesos = {
      'roe': 0.10,
      'roic': 0.10,
      'roa': 0.05,
      'margem_ebitda': 0.05,
      'cagr_lucros': 0.20,
      'pl': 0.06,
      'ev_ebitda': 0.06,
      'ev_ebit': 0.06,
      'p_vp': 0.03,
      'preco_ativo_circ': 0.03,
      'p_ebit': 0.03,
      'preco_cap_giro': 0.03,
      'div_brt_patr': 0.10,
      'pat_liq_ativ': 0.05,
      'liquidez': 0.05,
}

df_normalizado = pd.DataFrame(index=df_tratados.index)

is_melhor_maior = {
    'margem_ebitda': True,
    'roe': True,
    'roic': True,
    'roa': True,
    'liquidez': True,
    'cagr_lucros': True,
    'pat_liq_ativ': True,
    'pl': False,
    'p_vp': False,
    'ev_ebitda': False,
    'ev_ebit': False,
    'p_ebit': False,
    'preco_ativo_circ': False,
    'preco_cap_giro': False,
    'div_brt_patr': False
}

for col, peso in pesos.items():
    if col in df_tratados.columns:
        df_normalizado[col] =  normalizar_coluna(df_tratados[col], is_melhor_maior[col])
    else:
        print(f"Atenção: A coluna '{col}' não foi encontrada no dataframe original e será ignorada.")

df_normalizado['SCORE_FINAL'] = 0
for col, peso in pesos.items():
    if col in df_normalizado.columns:
        df_normalizado['SCORE_FINAL'] += df_normalizado[col] * peso

df_normalizado = df_normalizado.round(3)
df_normalizado.to_csv('indicadores_normalizados.csv', index=True)
df_normalizado.sort_values(by='SCORE_FINAL', ascending=False)

,roe,roic,roa,margem_ebitda,cagr_lucros,pl,ev_ebitda,ev_ebit,p_vp,preco_ativo_circ,p_ebit,preco_cap_giro,div_brt_patr,pat_liq_ativ,liquidez,SCORE_FINAL
Ativo,,,,,,,,,,,,,,,,
SBSP3,0.748,0.745,0.853,0.517,1.000,0.574,0.570,0.647,0.424,0.414,0.596,0.000,0.534,0.570,0.345,0.667
POMO4,0.771,0.634,0.865,0.192,1.000,0.534,0.431,0.542,0.344,1.000,0.459,0.538,0.529,0.547,0.551,0.662
VIVA3,0.692,0.788,0.988,0.323,0.762,0.490,0.326,0.411,0.355,1.000,0.340,0.559,0.664,0.775,1.000,0.662
PETR4,0.605,0.559,0.598,0.375,1.000,0.644,0.659,0.669,0.601,0.118,0.678,0.963,0.499,0.433,0.176,0.635
PETR3,0.605,0.559,0.598,0.375,1.000,0.631,0.659,0.669,0.588,0.131,0.666,0.987,0.499,0.433,0.176,0.634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENEV3,0.251,0.356,0.335,0.338,0.000,1.000,0.358,0.173,0.565,0.177,0.391,0.378,0.488,0.467,0.399,0.324
PCAR3,0.000,0.000,0.000,0.064,0.081,0.842,0.621,1.000,0.645,0.013,1.000,0.870,0.325,0.205,0.240,0.298
CSAN3,0.000,0.013,0.000,0.184,0.644,0.843,0.273,0.000,0.475,0.008,0.610,0.632,0.000,0.114,0.537,0.291


In [19]:
df_tratados['cagr_lucros'].mean()

np.float64(0.1378722891566265)

# Preços

In [ ]:
dt_ini, dt_fim =  '2020-01-01','2024-08-01'
df_precos = pd.DataFrame()
df_precos = yf.download(tickers, dt_ini, dt_fim)['Close']

In [ ]:
#validacao
serie = df_precos.isnull().sum()
serie = serie[serie>0]
ativos_nulos = list(serie.index)
ativos_nulos

Foram buscados precos a partir de 2023 de modo que todos os ativos tivessem precos pois alguns ativos , como ALOS3 não tinham precos antes desta data

In [ ]:
dt_ini, dt_fim =  '2023-01-01','2024-08-01'
df_precos = pd.DataFrame()
df_precos = yf.download(tickers, dt_ini, dt_fim)['Close']

# Nova seção

In [ ]:
sbsb3_indicators = df_normalizado.loc['LREN3']
average_indicators = df_normalizado.mean()

comparison_df = pd.DataFrame({
    'LREN3': sbsb3_indicators,
    'Average': average_indicators
})

# Calculate the difference between SBSP3 and the average
comparison_df['Difference'] = comparison_df['LREN3'] - comparison_df['Average']

# Filter for indicators where SBSP3 is above the average
superior_indicators = comparison_df[comparison_df['Difference'] > 0]

# Sort the results by the difference in descending order
superior_indicators_sorted = superior_indicators.sort_values(by='Difference', ascending=False)

display(superior_indicators_sorted)

In [11]:
from bs4 import BeautifulSoup
import re

ativo='BBAS3'
url ='https://investidor10.com.br/acoes/' + ativo
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

root = soup.select_one("#info_about")
container = (root.select_one("#table-indicators-company") if root else None) \
            or soup.select_one("#table-indicators-company") \
            or soup  # fallback

# monta {Título: Valor} a partir das células
kv = {}
for cell in container.select(".cell"):
    t = cell.select_one(".title")
    v = cell.select_one(".value .simple-value") or cell.select_one(".value")
    if t and v:
        kv[norm(t.get_text())] = norm(v.get_text(" "))



print({
    "Setor": pick("Setor"),
    "Segmento": pick("Segmento"),
})


{'Setor': 'Financeiro', 'Segmento': 'Bancos'}
